###### Credit : superdatascience.com

Business case : Classify possibly-churned customer using features of the data

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
import warnings
warnings.filterwarnings('ignore')

In [3]:
import tensorflow as tf

## Data

In [4]:
dataset = pd.read_csv('Churn_Modelling.csv')
X = dataset.iloc[:, 3:-1].values
y = dataset.iloc[:, -1].values

## Preprocessing

##### Missing Data

In [5]:
df = pd.DataFrame(data = X)

In [6]:
df.isnull().sum()

0    0
1    0
2    0
3    0
4    0
5    0
6    0
7    0
8    0
9    0
dtype: int64

##### Categorical Data Encoding - Gender

In [7]:
print(X)

[[619 'France' 'Female' ... 1 1 101348.88]
 [608 'Spain' 'Female' ... 0 1 112542.58]
 [502 'France' 'Female' ... 1 0 113931.57]
 ...
 [709 'France' 'Female' ... 0 1 42085.58]
 [772 'Germany' 'Male' ... 1 0 92888.52]
 [792 'France' 'Female' ... 1 0 38190.78]]


In [8]:
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
X[:, 2] = le.fit_transform(X[:, 2])

##### Categorical Data Encoding - Country

In [9]:
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder
ct = ColumnTransformer(transformers = [('encoder', OneHotEncoder(), [1])], remainder = 'passthrough')
X = np.array(ct.fit_transform(X))

In [10]:
print(X[:2, :])

[[1.0 0.0 0.0 619 0 42 2 0.0 1 1 1 101348.88]
 [0.0 0.0 1.0 608 0 41 1 83807.86 1 0 1 112542.58]]


##### Set Split

In [11]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 0)

##### Feature Scaling

In [12]:
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)

## Artificial Neural Network

##### Model

In [14]:
ann = tf.keras.models.Sequential()

ann.add(tf.keras.layers.Dense(units = 6, activation = 'relu')) 
ann.add(tf.keras.layers.Dense(units = 6, activation = 'relu'))

ann.add(tf.keras.layers.Dense(units = 1, activation = 'sigmoid')) 

ann.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = ['accuracy'])

##### Training

In [16]:
ann.fit(X_train, y_train, batch_size = 32, epochs = 100)

Epoch 1/100
250/250 [==============================] - 0s 838us/step - loss: 0.4142 - accuracy: 0.8329
Epoch 2/100
250/250 [==============================] - 0s 878us/step - loss: 0.4116 - accuracy: 0.8328
Epoch 3/100
250/250 [==============================] - 0s 892us/step - loss: 0.4096 - accuracy: 0.8328
Epoch 4/100
250/250 [==============================] - 0s 914us/step - loss: 0.4074 - accuracy: 0.8332
Epoch 5/100
250/250 [==============================] - 0s 711us/step - loss: 0.4050 - accuracy: 0.8326
Epoch 6/100
250/250 [==============================] - 0s 632us/step - loss: 0.4022 - accuracy: 0.8340
Epoch 7/100
250/250 [==============================] - 0s 637us/step - loss: 0.3990 - accuracy: 0.8345
Epoch 8/100
250/250 [==============================] - 0s 640us/step - loss: 0.3953 - accuracy: 0.8374
Epoch 9/100
250/250 [==============================] - 0s 627us/step - loss: 0.3914 - accuracy: 0.8382
Epoch 10/100
250/250 [==============================] - 0s 619us/step - l

250/250 [==============================] - 0s 627us/step - loss: 0.3380 - accuracy: 0.8633
Epoch 80/100
250/250 [==============================] - 0s 642us/step - loss: 0.3379 - accuracy: 0.8630
Epoch 81/100
250/250 [==============================] - 0s 626us/step - loss: 0.3380 - accuracy: 0.8634
Epoch 82/100
250/250 [==============================] - 0s 620us/step - loss: 0.3375 - accuracy: 0.8637
Epoch 83/100
250/250 [==============================] - 0s 621us/step - loss: 0.3377 - accuracy: 0.8615
Epoch 84/100
250/250 [==============================] - 0s 627us/step - loss: 0.3372 - accuracy: 0.8618
Epoch 85/100
250/250 [==============================] - 0s 621us/step - loss: 0.3372 - accuracy: 0.8634
Epoch 86/100
250/250 [==============================] - 0s 624us/step - loss: 0.3374 - accuracy: 0.8635
Epoch 87/100
250/250 [==============================] - 0s 623us/step - loss: 0.3373 - accuracy: 0.8640
Epoch 88/100
250/250 [==============================] - 0s 620us/step - loss:

##### Single Observation Prediction

###### Predict whether the customer with the following informations will leave the bank :
* Geography : France
* Credit Score : 600
* Gender : Male
* Age : 40 yrs old
* Tenure : 3 years
* Balance : usd 60,000
* Number of Products : 2
* Does this customer have a credit card? Yes
* Is this customer an active member? Yes
* Estimated Salary : $ 50,000

In [17]:
print(ann.predict(sc.transform([[1, 0, 0, 600, 1, 40, 3, 60000, 2, 1, 1, 50000]])))     # We get probability (sigmoid)

1/1 [==============================] - 0s 74ms/step
[[0.03777396]]


In [18]:
print(ann.predict(sc.transform([[1, 0, 0, 600, 1, 40, 3, 60000, 2, 1, 1, 50000]])) > 0.5)
# If you want a boolean value, 0.5 is a assigned threshold
# The customer will probabily stay

1/1 [==============================] - 0s 15ms/step
[[False]]


##### Prediction

In [19]:
y_pred = ann.predict(X_test)
y_pred = (y_pred > 0.5)
print(np.concatenate((y_pred.reshape(len(y_pred), 1), y_test.reshape(len(y_test), 1)), 1))

63/63 [==============================] - 0s 654us/step
[[0 0]
 [0 1]
 [0 0]
 ...
 [0 0]
 [0 0]
 [0 0]]


##### Evaluation

In [20]:
from sklearn.metrics import confusion_matrix, accuracy_score
cm = confusion_matrix(y_test, y_pred)
print(cm)
accuracy_score(y_test, y_pred)

[[1513   82]
 [ 195  210]]


0.8615